In [ ]:
import polars as pl
from sqlalchemy import create_engine, text
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

## 1. Carregamento dos dados

In [ ]:
def bd_conn_config(table):
    user = "mpllv"
    host = f"localhost/{table}"
    return f"postgresql+psycopg://{user}@{host}"

def get_data(conn, query):
    result = conn.execute(text(query))

    result_dict = result.mappings().all() 

    if len(result_dict) >= 1:
        organized_result = {key: [dict[key] for dict in result_dict] for key in result_dict[0]}
        return organized_result
    else: 
        return None
    
def get_data_params(conn, query, params):
    result = conn.execute(text(query), params)

    result_dict = result.mappings().all() 

    if len(result_dict) >= 1:
        organized_result = {key: [dict[key] for dict in result_dict] for key in result_dict[0]}
        return organized_result
    else: 
        return None


In [ ]:
engine = create_engine(bd_conn_config('paipe-test'))
conn = engine.connect()

query_train = 'select * from train_houses'
query_test = 'select * from test_houses'

train_df = pl.DataFrame(get_data(conn, query_train)).with_columns(
    pl.col('MinTimeToNearestStation').cast(pl.Int64), pl.col('MaxTimeToNearestStation').cast(pl.Int64),
    pl.col('TotalFloorArea').cast(pl.Int64), pl.col('BuildingYear').cast(pl.Int64), 
    pl.col('CoverageRatio').cast(pl.Int64), pl.col('FloorAreaRatio').cast(pl.Int64)
)

conn.close()

In [ ]:
print(train_df.shape)
train_df.head()

## 2. Preparação e limpeza dos dados

In [ ]:
train_df.describe()

In [ ]:
# Script para verificação das colunas a serem retiradas
# Retirar colunas com > 30% dos valores nulos
summary_train = train_df.describe()

data_size = len(train_df)

trim_row = []
trim_column = []

columns = summary_train.columns
columns.remove('statistic')
columns

for column in columns:
    nulls = int(summary_train[1, column])

    if nulls > (data_size * 0.3):
        trim_column.append(column)

print('columns to trim: ', trim_column)

In [ ]:
# Verificação dos tipos de propriedade
train_df.select('Region').group_by('Region').len()

In [ ]:
trim_column.remove('TotalFloorArea')
trim_column.remove('Frontage')
trim_column.remove('Region')
trim_column.remove('UnitPrice')

In [ ]:
# Somente se enquadram colunas no critério dos 30%
train_df = train_df.drop(trim_column)

In [ ]:
# Preenchimento de valores nulos em colunas que é possível substituir por 0
train_df = train_df.with_columns(
    pl.col('CoverageRatio').fill_null(0),
    pl.col('FloorAreaRatio').fill_null(0),
    pl.col('TotalFloorArea').fill_null(0),
    pl.col('Frontage').fill_null(0)
)

# Prenchimento dos valores nulos da coluna UnitPrice por -1
train_df = train_df.with_columns(
    pl.col('UnitPrice').fill_null(-1)
)

# Preenchimento dos valores nulos na coluna "CityPlanning" e "Region" por 'Other'
train_df = train_df.with_columns(pl.col('CityPlanning').fill_null('Other'),
                                 pl.col('Region').fill_null('Other'))

In [ ]:
# Boxplot inicial para análise de outliers
fig = px.box(train_df.to_pandas(), x = 'TradePrice')
fig.show()

In [ ]:
# Verifica-se que o limite superior do boxplot é 100M e representam menos de 10% dos dados
len(train_df.filter(pl.col('TradePrice') > 100000000))

In [ ]:
# Retira-se os outliers
train_df = train_df.filter(pl.col('TradePrice') <= 100000000)

In [ ]:
train_df.describe()

In [ ]:
# Substituição do trimestre pelo semestre e transformação da variável year em categórica
train_df = train_df.with_columns(pl.col('Quarter').cast(pl.String).str.replace_all('2', '1').str.replace_all(
    '4', '2').str.replace_all('3', '2'), 
    pl.col('Year').cast(pl.String)).rename({'Quarter': 'Semester'})

# A variável Year e Semester podem ser agrupadas e substituir a Period
train_df = train_df.replace_column(-4, train_df.with_columns(
    pl.concat_str([pl.col('Year'), pl.col('Semester')], separator='_').alias('YearSemester')
    ).select('YearSemester').to_series(0)).drop(['Year', 'Semester'])

# A variável TimeToNearestStation é categórica, mas pode ser substituída pela média do Min e Max
train_df = train_df.replace_column(7, train_df.with_columns(
    mean_time=pl.mean_horizontal("MinTimeToNearestStation", "MaxTimeToNearestStation").cast(pl.Int64).cast(pl.String)
    ).select('mean_time').rename(
    {'mean_time': 'TimeToNearestStation'}).to_series(0)
    ).with_columns(
    pl.col('TimeToNearestStation').cast(pl.Int64)
    ).drop(['MinTimeToNearestStation', 'MaxTimeToNearestStation'])

# Todas as observações são da Prefecture "Tokyo" e a MunicipalityCode já é representada pela Municipality
train_df = train_df.drop(['MunicipalityCode', 'Prefecture'])

# As colunas BuildingYear, Structure e Use ainda possuem um grande número de valores nulos
train_df = train_df.drop(['BuildingYear', 'Structure', 'Use'])

# Substituição das linhas sem informação na coluna DistrictName por Other
train_df = train_df.with_columns(pl.col('DistrictName').str.replace_all('(No Address)', 
                                 'Other', literal = True).fill_null('Other'))

# Tranformação da coluna FrontageIsGreaterFlag
train_df = train_df.with_columns(pl.col('FrontageIsGreaterFlag').cast(pl.String).str.replace_all(
    'false', '0').str.replace_all('true', '1').cast(pl.Int64))

In [ ]:
# Sumário das colunas com valores nulos restantes
remaining_null_columns = []
for item in train_df.describe().columns:
    if item != 'statistic':
        if int(train_df.describe()[1, item]) > 0:
            remaining_null_columns.append(item)
remaining_null_columns

In [ ]:
train_df.select(
    ['DistrictName', 'Type', 'NearestStation']).group_by(['DistrictName', 'Type']).max().filter(pl.col('NearestStation').is_not_null())

In [ ]:
# Adição de uma coluna de índice para facilitar a etapa seguinte
train_df.insert_column(0, pl.Series('index', list(range(len(train_df)))))

# Preenchimento dos valores faltantes com base em diferentes estratégias de acordo com a variável

# NearestStation: mais frequente dentre o distrito e tipo de propriedade
station_filler = train_df.select(
    ['DistrictName', 'Type', 'NearestStation']).group_by(
    ['DistrictName', 'Type']).max().filter(pl.col('NearestStation').is_not_null())

missing_stations = list(train_df.filter(pl.col('NearestStation').is_null())['index'])

for index in missing_stations:
    temp = train_df[index]
    filler = station_filler.filter((pl.col('DistrictName') == temp[0, 'DistrictName']) & 
                                        (pl.col('Type') == temp[0, 'Type']))
    if len(filler) > 0:
        train_df[index, 'NearestStation'] = filler[0, 'NearestStation']
    else:
        train_df[index, 'NearestStation'] = temp[0, 'DistrictName']

# TimeToNearestStation: média das propriedades com a mesma estação mais próxima
timeStation_filler = train_df.select(['DistrictName', 'NearestStation', 'TimeToNearestStation']).group_by(
    'DistrictName', 'NearestStation').mean().with_columns(pl.col('TimeToNearestStation').cast(pl.Int64))

missing_timeStation = list(train_df.filter(pl.col('TimeToNearestStation').is_null())['index'])

for index in missing_timeStation:
    temp = train_df[index]
    filler = timeStation_filler.filter(
            pl.col('NearestStation') == temp[0, 'NearestStation'])[0, 'TimeToNearestStation']
    train_df[index, 'TimeToNearestStation'] = filler

# Subsituição dos valores nulos em TimeToNearestStation por -1
train_df = train_df.with_columns(pl.col('TimeToNearestStation').fill_null(-1))


In [ ]:
# Sem valores nulos restantes
train_df.describe()

In [ ]:
# Retirando colunas que não serão utilizadas na regressão
train_df = train_df.drop(['DistrictName', 'NearestStation'])

## 3. Análise exploratória

In [ ]:
pd_train = train_df.drop('index').to_pandas()

In [ ]:
fig = px.box(pd_train, x = 'TradePrice')
fig.show()

In [ ]:
fig = px.box(pd_train, x = 'Type', y = 'TradePrice', color = 'Type')
fig.show()

In [ ]:
corr_train = pd_train.drop(columns = ['Type', 'Region',  'Municipality', 'CityPlanning', 'YearSemester']).corr()
fig = px.imshow(corr_train, x = corr_train.columns, y = corr_train.index, text_auto=True, aspect=True)
fig.show()

In [ ]:
x_prep_train = pd_train.iloc[:, :-1]
y_train = pd_train.iloc[:, -1].values

# Conversão das variáveis categóricas em dummies
x_train = pd.get_dummies(x_prep_train, 
                         columns=['Type','Region', 'Municipality', 'CityPlanning', 'YearSemester'], dtype=int).values


In [ ]:
x_train.shape

## 4. Criação do modelo

In [ ]:
grid_params = {'n_estimators': [150, 175, 200, 225, 250, 275, 300],
               'criterion': ['squared_error'], 
               'min_samples_split': [5, 10, 15, 20, 25], 
               'max_features': ['sqrt', 'log2'], 
               'bootstrap': [True, False],
               'oob_score': [True, False]}

rfr = GridSearchCV(estimator=RandomForestRegressor(random_state=123), param_grid=grid_params, n_jobs=8, 
                   cv=2, verbose=2)
rfr.fit(x_train, y_train)

In [ ]:
y_pred = rfr.predict(x_train)

print('Best parameters: ', rfr.best_params_)
print('Best score: ', rfr.best_score_)
print('RMSLE: ', metrics.root_mean_squared_log_error(y_train, y_pred))
print('MAPE: ', metrics.mean_absolute_percentage_error(y_train, y_pred))

## 5. Aplicação do modelo aos dados de teste

In [ ]:
engine = create_engine(bd_conn_config('paipe-test'))
conn = engine.connect()

query_train = 'select * from train_houses'
query_test = 'select * from test_houses'

test_df = pl.DataFrame(get_data(conn, query_test)).with_columns(
    pl.col('MinTimeToNearestStation').cast(pl.Int64), pl.col('MaxTimeToNearestStation').cast(pl.Int64),
    pl.col('TotalFloorArea').cast(pl.Int64), pl.col('BuildingYear').cast(pl.Int64), 
    pl.col('CoverageRatio').cast(pl.Int64), pl.col('FloorAreaRatio').cast(pl.Int64)
)

conn.close()

In [ ]:
# Preparação dos dados de teste com a mesma metodologia utilizada nos dados de treino

test_df = test_df.drop(trim_column).with_columns(
    pl.col('CoverageRatio').fill_null(0),
    pl.col('FloorAreaRatio').fill_null(0),
    pl.col('TotalFloorArea').fill_null(0),
    pl.col('Frontage').fill_null(0)
    ).with_columns(
    pl.col('UnitPrice').fill_null(-1)
    ).with_columns(
        pl.col('CityPlanning').fill_null('Other'),
        pl.col('Region').fill_null('Other'))

test_df = test_df.with_columns(pl.col('Quarter').cast(pl.String).str.replace_all('2', '1').str.replace_all(
    '4', '2').str.replace_all('3', '2'), 
    pl.col('Year').cast(pl.String)).rename({'Quarter': 'Semester'})

test_df = test_df.replace_column(-4, test_df.with_columns(
    pl.concat_str([pl.col('Year'), pl.col('Semester')], separator='_').alias('YearSemester')
    ).select('YearSemester').to_series(0)).drop(['Year', 'Semester'])

test_df = test_df.replace_column(7, test_df.with_columns(
    mean_time=pl.mean_horizontal("MinTimeToNearestStation", "MaxTimeToNearestStation").cast(pl.Int64).cast(pl.String)
    ).select('mean_time').rename(
    {'mean_time': 'TimeToNearestStation'}).to_series(0)
    ).with_columns(
    pl.col('TimeToNearestStation').cast(pl.Int64)
    ).drop(['MinTimeToNearestStation', 'MaxTimeToNearestStation'])

test_df = test_df.drop(['MunicipalityCode', 'Prefecture','BuildingYear', 'Structure', 'Use'])

test_df = test_df.with_columns(pl.col('DistrictName').str.replace_all('(No Address)', 
        'Other', literal = True).fill_null('Other'))

test_df = test_df.with_columns(
        pl.col('FrontageIsGreaterFlag').cast(pl.String).str.replace_all(
        'false', '0').str.replace_all('true', '1').cast(pl.Int64)
    )

test_df.insert_column(0, pl.Series('index', list(range(len(test_df)))))

missing_stations = list(test_df.filter(pl.col('NearestStation').is_null())['index'])
for index in missing_stations:
    temp = test_df[index]
    filler = station_filler.filter((pl.col('DistrictName') == temp[0, 'DistrictName']) & 
                                        (pl.col('Type') == temp[0, 'Type']))
    if len(filler) > 0:
        test_df[index, 'NearestStation'] = filler[0, 'NearestStation']
    else:
        test_df[index, 'NearestStation'] = temp[0, 'DistrictName']

missing_timeStation = list(test_df.filter(pl.col('TimeToNearestStation').is_null())['index'])
for index in missing_timeStation:
    temp = test_df[index]
    filler = timeStation_filler.filter(
            pl.col('NearestStation') == temp[0, 'NearestStation'])[0, 'TimeToNearestStation']
    test_df[index, 'TimeToNearestStation'] = filler

test_df = test_df.with_columns(pl.col('TimeToNearestStation').fill_null(-1)
                                 ).drop(['DistrictName', 'NearestStation'])

In [ ]:
pd_test = test_df.drop('index').to_pandas()
x_prep_test = pd_test.iloc[:, :-1]
x_test = pd.get_dummies(x_prep_test, columns=['Type', 'Region', 'Municipality', 'CityPlanning', 'YearSemester'], dtype=int).values
y_test = rfr.predict(x_test)

In [ ]:
pd_test['TradePrice'] = y_test

In [ ]:
final_df_train = train_df.drop('index').hstack([pl.Series('Origin', ['Train'] * len(train_df))])

final_df_test = pl.from_pandas(pd_test)
final_df_test = final_df_test.hstack([pl.Series('Origin', ['Test'] * len(final_df_test))])

combined = final_df_test.with_columns(pl.col('TradePrice').cast(pl.Int64)).vstack(final_df_train)

In [ ]:
combined = final_df_test.with_columns(pl.col('TradePrice').cast(pl.Int64)).vstack(final_df_train)

In [ ]:
fig = px.box(combined.to_pandas(), x = 'Type', y = 'TradePrice', color = 'Origin')
fig.show()

In [ ]:
final_df_test.write_csv('test_results.csv')